In [106]:
def read(filename):
    with open(filename, 'r') as f:
        N, K = map(int, f.readline().split())
        d = [0] + list(map(float, f.readline().split()))
        t = np.zeros((N + 1, N + 1))
        for i in range(N + 1):
            t[i] = list(map(float, f.readline().split()))

    return N, K, d, t


import numpy as np

def cal_cost(router, t, d):
    cost = [0 for i in range(len(router))]
    for i in range(len(router)):
        for j in range(len(router[i]) - 1):
            cost[i] += t[router[i][j]][router[i][j + 1]] + d[router[i][j + 1]]
    return cost

def UCV(visited):
        unvisited = []
        for pos in range(len(visited)):
            if visited[pos] == 0:
                unvisited.append(pos)
        return unvisited

def local_search_greedy1(file_data):
    N, K, d, t = read(file_data)
    router = []
    for i in range(K):
        router.append([0])

    visited = np.zeros(N + 1, dtype = int)
    visited[0] = 1      
        
    k = 0
    while sum(visited) != N + 1:
        start = router[k][-1]
        next_pos = 0
        next_time = 1000000
        unvisited = UCV(visited)
        for pos in unvisited:
            if t[start][pos] + d[pos] < next_time:
                next_time = t[start][pos] + d[pos]
                next_pos = pos
        router[k].append(next_pos)
        visited[next_pos] = 1
        k = (k + 1) % K
    
    for i in range(K):
        router[i].append(0)
    cost = [0 for i in range(K)]
    for i in range(K):
        for j in range(len(router[i]) - 1):
            cost[i] += t[router[i][j]][router[i][j + 1]] + d[router[i][j + 1]]
    history_count = 0
    while True:
        max_cost = max(cost)
        max_index = [i for i, j in enumerate(cost) if j == max_cost]
        router_neighborhood = []

        swap1 = np.random.choice(max_index)
        swap2 = np.random.randint(0, K)
        while swap2 == swap1:
            swap2 = np.random.randint(0, K)
        
        #a1 != b1
        a1 = np.random.randint(1, len(router[swap1]) - 2)
        b1 = np.random.randint(a1 + 1, len(router[swap1]) - 1)

        a2 = np.random.randint(1, len(router[swap2]) - 2)
        b2 = np.random.randint(a2 + 1, len(router[swap2]) - 1)

        new_router1 = router[swap1][:a1] + router[swap2][a2:b2] + router[swap1][b1:]
        new_router2 = router[swap2][:a2] + router[swap1][a1:b1] + router[swap2][b2:]

        for i in range(K):
            if i == swap1:
                router_neighborhood.append(new_router1)
            elif i == swap2:
                router_neighborhood.append(new_router2)
            else:
                router_neighborhood.append(router[i])
        
        cost_neighborhood = cal_cost(router_neighborhood, t, d)
        new_max_cost = max(cost_neighborhood)
        if new_max_cost < max_cost:
            router = router_neighborhood
            cost = cost_neighborhood
            history_count = 0
        else:
            history_count += 1
        if history_count == 1000:
            break
        

    print(K)
    for sub in router:
        print(len(sub))
        for pos in sub:
            print(pos, end = ' ')
        print()
    return router, cost


In [120]:
file_data = "data\\data_10_4.txt"
router, cost = local_search_greedy1(file_data)
print(max(cost))

4
4
0 2 3 0 
4
0 6 9 0 
5
0 4 7 1 0 
5
0 10 8 5 0 
1722.0
